In [1]:
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

In [3]:
mnist_path = '../data'
model_path = './lenet_mnist_model.bin'

# LeNet Model definition
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# MNIST Test dataset and dataloader declaration
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(mnist_path, train=False, download=True, transform=transforms.Compose([
            transforms.ToTensor(),
            ])), 
        batch_size=1, shuffle=True)

# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (torch.cuda.is_available()) else "cpu")

# Initialize the network
model = Net().to(device)

# Load the pretrained model
model.load_state_dict(torch.load(model_path, map_location='cpu'))

# Set the model in evaluation mode. In this case this is for the Dropout layers
model.eval()

CUDA Available:  False


Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [27]:
# FGSM attack code
def fgsm_attack(model, data, target, epsilon=0.1, critrion=F.nll_loss):
    
    # Set requires_grad attribute of tensor. Important for Attack
    data.requires_grad = True
    
    # Forward pass the data through the model
    output = model(data)

    # Calculate the loss
    loss = criterion(output, target)

    # Zero all existing gradients
    model.zero_grad()

    # Calculate gradients of model in backward pass
    loss.backward()

    # Collect datagrad
    data_grad = data.grad.data
    
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = data + epsilon*sign_data_grad
    
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    
    # Return the perturbed image
    return perturbed_image

In [48]:
# BIM attack code
def bim_attack(model, data, target, alpha = 0.05, nb_iter = 3, epsilon=0.05, critrion=F.nll_loss):
    
    eta = 0
    for i in range(nb_iter):
        # Call FGSM Attack
        perturbed_data = fgsm_attack(model, (data+eta).detach(), target, alpha, critrion)
        
        # Clip intermediate results
        eta = perturbed_data - data
        eta = torch.clamp(eta, -epsilon, epsilon)

    # Return the perturbed image
    return data + eta

In [50]:
def test():

    # Accuracy counter
    correct0 = 0
    correct1 = 0

    # Loop over all examples in test set
    for data, target in test_loader:

        # Send the data and label to the device
        data, target = data.to(device), target.to(device)

        # Call Attack Method
        perturbed_data = fgsm_attack(model, data, target)
        # perturbed_data = bim_attack(model, data, target)

        # Classify the image
        init_output = model(data)
        final_output = model(perturbed_data)

        # Check for success
        init_pred = init_output.max(1, keepdim=True)[1] # get the index of the max log-probability
        final_pred = final_output.max(1, keepdim=True)[1]
        
        if init_pred.item() == target.item():
            correct0 += 1
        
        if final_pred.item() == target.item():
            correct1 += 1

    # Calculate final accuracy for this epsilon
    init_acc = correct0/float(len(test_loader))
    final_acc = correct1/float(len(test_loader))
    print("Accuracy: {} -> {}".format(init_acc, final_acc))
    
test()

Accuracy: 0.981 -> 0.851
